In [7]:
import base64
import json
import re
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from elevenlabs.client import ElevenLabs
from elevenlabs.play import play

load_dotenv()

# -----------------------------
# Function Definitions
# -----------------------------

def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def generate_educational_json(img_base64: str) -> dict:
    """
    Analyze uploaded image and generate neutral educational description.
    Returns: caption, tags, scam_type, category.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are an educational assistant. Analyze the given image and return a short, neutral description. "
                "Return caption, tags, scam_type, and category in JSON format ONLY. "
                "Do NOT include humor, emojis, or extra text. "
                "If scam_type is not applicable, set it to 'Unknown'."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe this image."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
            ]
        }
    ]

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.3
    )

    raw_output = response.choices[0].message.content
    match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    json_text = match.group() if match else "{}"

    try:
        result = json.loads(json_text)
    except json.JSONDecodeError:
        result = {"caption": raw_output, "tags": [], "scam_type": "Unknown", "category": "Unknown"}

    # Ensure all keys exist
    result = {
        "caption": result.get("caption", ""),
        "tags": result.get("tags", []),
        "scam_type": result.get("scam_type", "Unknown"),
        "category": result.get("category", "Unknown")
    }
    
    return result

def generate_narration_from_json(scam_json: dict) -> str:
    """
    Converts the educational JSON content into TTS-ready narration, 
    highlighting why it is a scam and identifying key words or cues that indicate the scam.
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", 
         "You are a narration assistant for educational videos. "
         "Transform the scam JSON content into clear, neutral, and concise narration suitable for TTS. "
         "Explain why the content is obviously a scam, and highlight key words, phrases, or cues that indicate the scam. "
         "Do NOT include humor, emojis, or formatting."),
        ("user", 
         "Use the following scam JSON to generate the narration:\n\n{scam_json}")
    ])
    
    messages = prompt_template.format_messages(scam_json=json.dumps(scam_json, indent=2))
    response = llm.invoke(messages)
    
    return response.content.strip()



def generate_and_play_audio(narration: str, voice_id: str = "pNInz6obpgDQGcFmaJgB"):
    """
    Generates audio from narration using ElevenLabs TTS and plays it.
    """
    elevenlabs = ElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))
    audio = elevenlabs.text_to_speech.convert(
        text=narration,
        voice_id=voice_id,
        model_id="eleven_multilingual_v2",
        output_format="mp3_44100_128",
    )
    play(audio)

def generate_starter_frame(script_text: str, output_path="starter_frame.png"):
    """
    Generates a single neutral starter frame for the educational video
    showing a representation of the scenario.
    """
    client = OpenAI()
    prompt = (
        f"Create a neutral, realistic image representing the following educational scenario: '{script_text}'. "
        f"No humor, no memes, no bright colors. "
        f"Keep it coherent and factual, suitable as a starter frame for an educational video."
    )
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        response_format="b64_json"
    )
    image_b64 = response.data[0].b64_json
    with open(output_path, "wb") as f:
        f.write(base64.b64decode(image_b64))
    return output_path

# -----------------------------
# Main Pipeline
# -----------------------------
def main():
    img_base64 = encode_image_to_base64("user_upload.jpg")
    educ_json = generate_educational_json(img_base64)
    print("✅ Educational JSON:")
    print(json.dumps(educ_json, indent=4))

    narration = generate_narration_from_json(scam_json)
    print("\n🎤 Narration Text for 11Labs:\n", narration)

    starter_frame_file = generate_starter_frame(narration)
    print("🖼️ Starter Frame saved at:", starter_frame_file)
    
    generate_and_play_audio(narration)

if __name__ == "__main__":
    main()


✅ Educational JSON:
{
    "caption": "A text message claiming to be from HSBC about unusual account activity.",
    "tags": [
        "SMS",
        "scam",
        "HSBC",
        "phishing"
    ],
    "scam_type": "Phishing",
    "category": "Fraud"
}

🎤 Narration Text for 11Labs:
 This is a narration about a fake bank alert email, which is a common type of phishing scam. Phishing scams typically involve fraudulent emails that appear to be from legitimate institutions, such as banks. 

Key indicators of this scam include the use of urgent language, requests for personal information, and links that lead to unofficial websites. 

Always verify the sender's email address and look for signs of poor grammar or spelling mistakes, which are often present in these scams. Remember, legitimate banks will never ask for sensitive information via email. Stay vigilant and protect your financial information.
🖼️ Starter Frame saved at: starter_frame.png


In [ ]:
import base64
from runwayml import RunwayML

client = RunwayML()

image = './example.png'

# Encode image to a data URI
with open(image, "rb") as f:
    # Read the file as a base64 encoded string
    base64_image = base64.b64encode(f.read()).decode("utf-8")
    # Format it as a data URI
    # We're using `image/png` here because the input is a PNG.
    data_uri = f"data:image/png;base64,{base64_image}"

# Create a new image-to-video task using the "gen4_turbo" model
try:
  task = client.image_to_video.create(
    model='gen4_turbo',
    # Point this at your own image file
    prompt_image=data_uri,
    prompt_text='A timelapse on a sunny day with clouds flying by',
    ratio='1280:720',
    duration=5,
  ).wait_for_task_output()

  print('Task complete:', task)
except runwayml.TaskFailedError as e:
  print('The video failed to generate.')
  print(e.task_details)